In [4]:
import pandas as pd
import numpy as np
import os

for dirname, _, files in os.walk(r"C:\Users\Manasa\Desktop\ml\nlp_project"):
    for file in files:
        if file == 'semeval2017_train.tsv':
            tsv_file = pd.read_table(os.path.join(r"C:\Users\Manasa\Desktop\ml\nlp_project\SEMEVAL2017", file))


tsv_file

,question,answer,label
0,from dubai to qatar i am currently working her...,if you are single then its ok you can enjoy,0
1,from dubai to qatar i am currently working her...,depends on where the accommodation is how many...,1
2,from dubai to qatar i am currently working her...,if the company is from oil and gas industry or...,1
3,from dubai to qatar i am currently working her...,transport in the city is a nightmare,1
4,from dubai to qatar i am currently working her...,and life her is so relaxed that bachelors are ...,0
...,...,...,...
36193,diamond earrings or gold earrings hi guys i am...,have you not heard the statements like diamond...,1
36194,diamond earrings or gold earrings hi guys i am...,go for jewelry that is yellow gold and without...,1
36195,diamond earrings or gold earrings hi guys i am...,of course gold,1
36196,diamond earrings or gold earrings hi guys i am...,for investment both are ok if you invest like ...,1


In [5]:
# converting tsv file into csv
tsv_file.to_csv('semeval_.csv',index=False)

data = pd.read_csv(r"C:\Users\Manasa\Desktop\ml\nlp_project\semeval_.csv")
data

,question,answer,label
0,from dubai to qatar i am currently working her...,if you are single then its ok you can enjoy,0
1,from dubai to qatar i am currently working her...,depends on where the accommodation is how many...,1
2,from dubai to qatar i am currently working her...,if the company is from oil and gas industry or...,1
3,from dubai to qatar i am currently working her...,transport in the city is a nightmare,1
4,from dubai to qatar i am currently working her...,and life her is so relaxed that bachelors are ...,0
...,...,...,...
36193,diamond earrings or gold earrings hi guys i am...,have you not heard the statements like diamond...,1
36194,diamond earrings or gold earrings hi guys i am...,go for jewelry that is yellow gold and without...,1
36195,diamond earrings or gold earrings hi guys i am...,of course gold,1
36196,diamond earrings or gold earrings hi guys i am...,for investment both are ok if you invest like ...,1


In [7]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop = stopwords.words('english')
data['question'] = data['question'].apply(lambda x: [item for item in x.split() if item not in stop])
data['answer'] = data['answer'].apply(lambda x: [item for item in x.split() if item not in stop])

data

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Manasa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,question,answer,label
0,"[dubai, qatar, currently, working, dubai, got,...","[single, ok, enjoy]",0
1,"[dubai, qatar, currently, working, dubai, got,...","[depends, accommodation, many, people, live, k...",1
2,"[dubai, qatar, currently, working, dubai, got,...","[company, oil, gas, industry, working, industr...",1
3,"[dubai, qatar, currently, working, dubai, got,...","[transport, city, nightmare]",1
4,"[dubai, qatar, currently, working, dubai, got,...","[life, relaxed, bachelors, excluded, many, ven...",0
...,...,...,...
36193,"[diamond, earrings, gold, earrings, hi, guys, ...","[heard, statements, like, diamonds, forever, d...",1
36194,"[diamond, earrings, gold, earrings, hi, guys, ...","[go, jewelry, yellow, gold, without, gems, sto...",1
36195,"[diamond, earrings, gold, earrings, hi, guys, ...","[course, gold]",1
36196,"[diamond, earrings, gold, earrings, hi, guys, ...","[investment, ok, invest, like, gold, buy, gold...",1


In [30]:
len(data['question'].unique())

4877

## Model

In [24]:
!pip install -U sentence-transformers

In [34]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
question1 = data['question'][0]
question2 = data['question'][36193]
answers = data['answer'][0]

q_embedding1 = model.encode(question1)
q_embedding2 = model.encode(question2)
a_embeddings = model.encode(answers)

print([len(q_embedding1), len(q_embedding2)])
print(len(a_embeddings))

[384, 384]


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

In [ ]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def SiamesNet():
    
    set_seed(33)
    
    left_input = Input((sequence_length, len(features)))
    right_input = Input((sequence_length, len(features)))

    lstmnet = Sequential([
        BatchNormalization(),
        LSTM(128, activation='relu', return_sequences=True, 
             input_shape=(sequence_length, len(features))),
        LSTM(32, activation='relu')
    ])

    encoded_l = lstmnet(left_input)
    encoded_r = lstmnet(right_input)

    L1_layer = Lambda(lambda tensor: K.abs(tensor[0] - tensor[1]))

    L1_distance = L1_layer([encoded_l, encoded_r])
    drop = Dropout(0.2)(L1_distance)
    prediction = Dense(1,activation='sigmoid')(drop)
    model = Model(inputs=[left_input,right_input],outputs=prediction)
    
    return model